In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

In [2]:
# import Gelderman SOD-labeled data
g_data = pd.read_pickle('./data_processed')
print(len(g_data))
display(g_data.head())

249


,TDS,PMI_days,log_PMI_days,est_weight_lb,est_stature_in,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,fall,spring,summer,log_ADD_thres0,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std
0,9.0,15.0,2.772589,180.0,64.000000,3.0,3.0,3.0,0,0,0,4.315902,73.881146,65.115868,32.350972,0.000000,0.000000,0.000000,0.00000,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361
1,13.0,79.0,4.382027,516.0,73.000039,5.0,5.0,3.0,0,1,0,7.190920,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.000000,0.00000,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009
2,15.0,164.0,5.105945,516.0,73.000039,6.0,6.0,3.0,0,0,1,8.175567,3552.068684,3552.068684,3505.354100,3209.104100,2617.501645,1785.331086,30.05625,27.174,1.174,71.959,6.483,28.471,0.317,63.152,0.949,26.955,1.031,72.737,6.045,26.864,1.680,67.719,7.432,18.606,5.250,63.088,13.413,12.155,8.104,73.635,12.741
3,13.0,50.0,3.931826,235.0,73.000039,5.0,4.0,4.0,0,0,0,5.764677,317.835928,268.610789,126.581344,30.420863,0.000000,0.000000,0.00000,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603
4,6.0,11.0,2.484907,170.0,69.000037,2.0,2.0,2.0,0,0,0,4.258335,69.692188,54.013368,39.090451,15.414583,0.000000,0.000000,0.00000,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609


In [4]:
# natural log transform PMI and ADD
g_data['log_PMI_days'] = np.log(g_data.PMI_days+0.001)
g_data['log_ADD_thres0'] = np.log(g_data.ADD_thres0+0.001)
g_data.head()

,new_id,donor_date,correct_img_date,date_placed_ARF,PMI_days,age_at_death,est_weight_lb,est_stature_in,img_head,true_SOD_G_head,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,sex_male,month,season_of_recovery,fall,spring,summer,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,log_PMI_days,log_ADD_thres0
0,004,00400124,2018-01-24,2018-01-09,15.0,77.0,180.0,64.000000,00400124.12.JPG,3.0,00400124.07.JPG,3.0,00400124.10.JPG,3.0,0,01,winter,0,0,0,73.881146,65.115868,32.350972,0.000000,0.000000,0.000000,0.00000,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,2.708117,4.302471
1,00b,00b00525,2016-05-25,2016-03-07,79.0,38.0,516.0,73.000039,00b00525.08.JPG,5.0,00b00525.04.JPG,5.0,00b00525.27.JPG,3.0,1,05,spring,0,1,0,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.000000,0.00000,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,4.369461,7.190167
2,00b,00b00818,2016-08-18,2016-03-07,164.0,38.0,516.0,73.000039,00b00818.07.JPG,6.0,00b00818.05.JPG,6.0,00b00818.11.JPG,3.0,1,08,summer,0,0,1,3552.068684,3552.068684,3505.354100,3209.104100,2617.501645,1785.331086,30.05625,27.174,1.174,71.959,6.483,28.471,0.317,63.152,0.949,26.955,1.031,72.737,6.045,26.864,1.680,67.719,7.432,18.606,5.250,63.088,13.413,12.155,8.104,73.635,12.741,5.099873,8.175286
3,00d,00d10116,2019-01-16,2018-11-27,50.0,73.0,235.0,73.000039,00d10116.07.JPG,5.0,00d10116.04.JPG,4.0,00d10116.06.JPG,4.0,1,01,winter,0,0,0,317.835928,268.610789,126.581344,30.420863,0.000000,0.000000,0.00000,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,3.912043,5.761538
4,011,01101210,2018-12-10,2018-11-29,11.0,90.0,170.0,69.000037,01101210.06.JPG,2.0,01101210.03.JPG,2.0,01101210.08.JPG,2.0,0,12,winter,0,0,0,69.692188,54.013368,39.090451,15.414583,0.000000,0.000000,0.00000,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,2.397986,4.244103


# PMI regression

In [3]:
# create X and y dataframes
X = g_data[['true_SOD_G_head', 'true_SOD_G_torso', 'true_SOD_G_limbs']]
display(X.head(2))
print(X.shape)

y = g_data[['log_PMI_days']]
display(y.head(2))
print(y.shape)

,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs
0,3.0,3.0,3.0
1,5.0,5.0,3.0


(249, 3)


,log_PMI_days
0,2.772589
1,4.382027


(249, 1)


In [4]:
# fit model
model1 = LinearRegression()
model1.fit(X, y)

LinearRegression()

In [5]:
# make predictions
y_pred = model1.predict(X)

In [6]:
# The coefficients
print("Coefficients: \n", model1.coef_)
# The mean squared error
print("MSE: %.2f" % mean_squared_error(y, y_pred))
print("RMSE: %.2f" % math.sqrt(mean_squared_error(y, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y, y_pred))

Coefficients: 
 [[-0.00704452  0.44791383  0.20842859]]
MSE: 1.01
RMSE: 1.00
Coefficient of determination: 0.29


In [7]:
g_data['pred_log_PMI'] = y_pred.copy()
g_data['pred_PMI'] = np.exp(y_pred).copy()
display(g_data.head())

,TDS,PMI_days,log_PMI_days,est_weight_lb,est_stature_in,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,fall,spring,summer,log_ADD_thres0,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,pred_log_PMI,pred_PMI
0,9.0,15.0,2.772589,180.0,64.000000,3.0,3.0,3.0,0,0,0,4.315902,73.881146,65.115868,32.350972,0.000000,0.000000,0.000000,0.00000,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,3.464766,31.968974
1,13.0,79.0,4.382027,516.0,73.000039,5.0,5.0,3.0,0,1,0,7.190920,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.000000,0.00000,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,4.346504,77.208109
2,15.0,164.0,5.105945,516.0,73.000039,6.0,6.0,3.0,0,0,1,8.175567,3552.068684,3552.068684,3505.354100,3209.104100,2617.501645,1785.331086,30.05625,27.174,1.174,71.959,6.483,28.471,0.317,63.152,0.949,26.955,1.031,72.737,6.045,26.864,1.680,67.719,7.432,18.606,5.250,63.088,13.413,12.155,8.104,73.635,12.741,4.787374,119.985848
3,13.0,50.0,3.931826,235.0,73.000039,5.0,4.0,4.0,0,0,0,5.764677,317.835928,268.610789,126.581344,30.420863,0.000000,0.000000,0.00000,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,4.107019,60.765321
4,6.0,11.0,2.484907,170.0,69.000037,2.0,2.0,2.0,0,0,0,4.258335,69.692188,54.013368,39.090451,15.414583,0.000000,0.000000,0.00000,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,2.815468,16.700989


In [8]:
print("RMSE (unlogged): %.2f" % math.sqrt(mean_squared_error(g_data['PMI_days'], g_data['pred_PMI'])))

RMSE (unlogged): 89.38


# ADD regression

In [9]:
# create X and y dataframes
X = g_data[['true_SOD_G_head', 'true_SOD_G_torso', 'true_SOD_G_limbs']]
display(X.head(2))
print(X.shape)

y = g_data[['log_ADD_thres0']]
display(y.head(2))
print(y.shape)

,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs
0,3.0,3.0,3.0
1,5.0,5.0,3.0


(249, 3)


,log_ADD_thres0
0,4.315902
1,7.190920


(249, 1)


In [10]:
# fit model
model2 = LinearRegression()
model2.fit(X, y)

LinearRegression()

In [11]:
# make predictions
y_pred = model2.predict(X)

In [12]:
# The coefficients
print("Coefficients: \n", model2.coef_)
# The mean squared error
print("MSE: %.2f" % mean_squared_error(y, y_pred))
print("RMSE: %.2f" % math.sqrt(mean_squared_error(y, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y, y_pred))

Coefficients: 
 [[0.14638713 0.43588311 0.30858424]]
MSE: 0.99
RMSE: 1.00
Coefficient of determination: 0.42


In [13]:
g_data['pred_log_ADD_thres0'] = y_pred.copy()
g_data['pred_ADD_thres0'] = np.exp(y_pred).copy()
display(g_data.head())

,TDS,PMI_days,log_PMI_days,est_weight_lb,est_stature_in,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,fall,spring,summer,log_ADD_thres0,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,pred_log_PMI,pred_PMI,pred_log_ADD_thres0,pred_ADD_thres0
0,9.0,15.0,2.772589,180.0,64.000000,3.0,3.0,3.0,0,0,0,4.315902,73.881146,65.115868,32.350972,0.000000,0.000000,0.000000,0.00000,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,3.464766,31.968974,5.827792,339.607958
1,13.0,79.0,4.382027,516.0,73.000039,5.0,5.0,3.0,0,1,0,7.190920,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.000000,0.00000,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,4.346504,77.208109,6.992332,1088.256734
2,15.0,164.0,5.105945,516.0,73.000039,6.0,6.0,3.0,0,0,1,8.175567,3552.068684,3552.068684,3505.354100,3209.104100,2617.501645,1785.331086,30.05625,27.174,1.174,71.959,6.483,28.471,0.317,63.152,0.949,26.955,1.031,72.737,6.045,26.864,1.680,67.719,7.432,18.606,5.250,63.088,13.413,12.155,8.104,73.635,12.741,4.787374,119.985848,7.574603,1948.085959
3,13.0,50.0,3.931826,235.0,73.000039,5.0,4.0,4.0,0,0,0,5.764677,317.835928,268.610789,126.581344,30.420863,0.000000,0.000000,0.00000,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,4.107019,60.765321,6.865034,958.177941
4,6.0,11.0,2.484907,170.0,69.000037,2.0,2.0,2.0,0,0,0,4.258335,69.692188,54.013368,39.090451,15.414583,0.000000,0.000000,0.00000,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,2.815468,16.700989,4.936937,139.342844


In [14]:
print("RMSE (unlogged): %.2f" % math.sqrt(mean_squared_error(g_data['ADD_thres0'], g_data['pred_ADD_thres0'])))

RMSE (unlogged): 1546.74


### Derive predicted PMI from predicted ADD
We will now go backwards, meaning deriving the PMI from the predicted ADD. Note, to align with Gelderman's study, a daily temperature below 0 degrees C is counted as 0 since decomposition stops below freezing. Usually, higher temps, faster decomposition, and so smaller PMI, while lower temps, slower decomposition, and so higher PMI. 

In [228]:
# import weather history
weather_df = pd.read_pickle('./../../temp_humidity_data/data/LCD/lcd_daily_avg.pkl')
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%Y-%m-%d', errors='coerce')
display(weather_df.head())
display(weather_df.info())

,date,HourlyDryBulbTemperature,HourlyRelativeHumidity
0,2011-01-01,11.472727,91.803030
1,2011-01-02,3.828571,64.628571
2,2011-01-03,-0.068750,57.812500
3,2011-01-04,3.084375,60.156250
4,2011-01-05,1.008333,83.216667


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4473 entries, 0 to 4472
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      4473 non-null   datetime64[ns]
 1   HourlyDryBulbTemperature  4473 non-null   float64       
 2   HourlyRelativeHumidity    4473 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 139.8 KB


None

In [229]:
# derive PMI from predicted ADD
df_dict = g_data.to_dict('records')
for row in df_dict:
    #print(row)
    discovery_date = row['correct_img_date']
    pred_ADD = row['pred_ADD_thres0']
    
    current_ADD = 0
    derived_PMI = 0
    current_date = discovery_date
    while (current_ADD < pred_ADD):
        temp = weather_df[weather_df.date == current_date]['HourlyDryBulbTemperature'].values[0]
        if temp < 0:
            current_ADD += 0
        else:
            current_ADD += temp
        
        derived_PMI += 1
        
        #print('date:'+str(start_date)+', current_ADD:'+str(current_ADD)+', temp:'+str(temp),  end="\n")
        current_date -= datetime.timedelta(days=1)
        
    row['PMI_from_pred_ADD_thres0'] = derived_PMI
    #print(derived_PMI)
    #break
    #print()

In [230]:
g_data2 = pd.DataFrame.from_dict(df_dict)
display(g_data2.head())
print(g_data2.shape)

,new_id,donor_date,correct_img_date,date_placed_ARF,PMI_days,age_at_death,est_weight_lb,est_stature_in,img_head,true_SOD_G_head,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,sex_male,month,season_of_recovery,fall,spring,summer,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,log_PMI_days,log_ADD_thres0,pred_log_PMI,pred_PMI,pred_log_ADD_thres0,pred_ADD_thres0,PMI_from_pred_ADD_thres0
0,004,00400124,2018-01-24,2018-01-09,15.0,77.0,180.0,64.000000,00400124.12.JPG,3.0,00400124.07.JPG,3.0,00400124.10.JPG,3.0,0,01,winter,0,0,0,73.881146,65.115868,32.350972,0.000000,0.000000,0.000000,0.00000,2.708117,4.302471,3.100459,22.208146,5.678407,292.483198,64
1,00b,00b00525,2016-05-25,2016-03-07,79.0,38.0,516.0,73.000039,00b00525.08.JPG,5.0,00b00525.04.JPG,5.0,00b00525.27.JPG,3.0,1,05,spring,0,1,0,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.000000,0.00000,4.369461,7.190167,4.209160,67.299999,7.025485,1124.940563,69
2,00b,00b00818,2016-08-18,2016-03-07,164.0,38.0,516.0,73.000039,00b00818.07.JPG,6.0,00b00818.05.JPG,6.0,00b00818.11.JPG,3.0,1,08,summer,0,0,1,3552.068684,3552.068684,3505.354100,3209.104100,2617.501645,1785.331086,30.05625,5.099873,8.175286,4.763511,117.156512,7.699025,2206.195037,85
3,00d,00d10116,2019-01-16,2018-11-27,50.0,73.0,235.0,73.000039,00d10116.07.JPG,5.0,00d10116.04.JPG,4.0,00d10116.06.JPG,4.0,1,01,winter,0,0,0,317.835928,268.610789,126.581344,30.420863,0.000000,0.000000,0.00000,3.912043,5.761538,4.172689,64.889737,6.922107,1014.454902,107
4,011,01101210,2018-12-10,2018-11-29,11.0,90.0,170.0,69.000037,01101210.06.JPG,2.0,01101210.03.JPG,2.0,01101210.08.JPG,2.0,0,12,winter,0,0,0,69.692188,54.013368,39.090451,15.414583,0.000000,0.000000,0.00000,2.397986,4.244103,2.201118,9.035106,4.688939,108.737713,20


(256, 34)


# Evaluate results

In [213]:
def msle(true_arr, pred_arr):
    log_differences = np.subtract(np.log(1 + true_arr), np.log(1 + pred_arr))
    squared_log_differences = np.square(log_differences)
    return np.mean(squared_log_differences)

In [234]:
# calculate MSE, RMSE, MSLE, RMSLE for PMI and ADD

# predicted PMI
mse_pmi = ((g_data2['PMI_days'] - g_data2['pred_PMI']) ** 2).mean()
rmse_pmi = math.sqrt(mse_pmi)
msle_pmi = msle(g_data2['PMI_days'], g_data2['pred_PMI'])
rmsle_pmi = math.sqrt(msle_pmi)
print('MSE:', mse_pmi, 'RMSE:', rmse_pmi, 'MSLE:', msle_pmi, 'RMSLE:', rmsle_pmi)
print()

# PMI derived from predicted ADD
mse_pmi_derived = ((g_data2['PMI_days'] - g_data2['PMI_from_pred_ADD_thres0']) ** 2).mean()
rmse_pmi_derived = math.sqrt(mse_pmi_derived)
msle_pmi_derived = msle(g_data2['PMI_days'], g_data2['PMI_from_pred_ADD_thres0'])
rmsle_pmi_derived = math.sqrt(msle_pmi_derived)
print('MSE:', mse_pmi_derived, 'RMSE:', rmse_pmi_derived, 'MSLE:', msle_pmi_derived, 'RMSLE:', rmsle_pmi_derived)

MSE: 9434.316502764637 RMSE: 97.13040977348257 MSLE: 1.165117073530402 RMSLE: 1.0794058891493978

MSE: 11040.20703125 RMSE: 105.07238948101447 MSLE: 1.2888551689806282 RMSLE: 1.1352775735390126
